In [43]:
# ClassifierV6 notebook updated by Joe V for more advanced data cleaning for use in fast-bert scripts.

# Jacob, addition
import nltk
nltk.download('stopwords')

# originals
import pandas as pd
import numpy as np
import csv 
import string
import nltk 
import re 
import time
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import os
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
import sys
import warnings
from os import path
import ast
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.problem_transform import ClassifierChain
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, average_precision_score, precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import zero_one_loss
from sklearn.metrics import jaccard_score
import sklearn.metrics as skm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import time
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from skmultilearn.adapt import MLkNN
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from skmultilearn.adapt import BRkNNaClassifier
from sklearn.linear_model import RidgeClassifierCV
import emoji

pd.options.display.max_seq_items = 2000
pd.options.display.max_colwidth = 90
pd.options.display.max_rows = 999

# project
proj_name       = "rxjava"
working_dir     = "./"


# input
binaryBodyTitle = working_dir + "data/" + proj_name + '_' + "binaryBodyTitle_without_filter.csv"
print( "Input Binary File: " + binaryBodyTitle )


# output
output_root_dir = working_dir + "goal format/"  
out_path        = output_root_dir + proj_name + '_' 

print( "Output path      : " + out_path )

dfTeste_output  = out_path + "dfTeste.csv"
print( "dfTeste output   : " + dfTeste_output )
# UNUSED: loggingFile  = out_path + "TitleBody3GramTFIDF.txt"

# !! see In[45] for the definitions of these output paths. They are kept here for posterity but must be defined there
# tfIDFoutputFile        = out_path + 'tfIDFoutputFile'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'
# classifierFeatureInput = out_path + 'train_file_test'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'
# termFrequencyTop50     = out_path + 'termFrequencyTop50'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'
# predictions_result     = out_path + 'predict_file_'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'
# probabilities_result   = out_path + 'probability_file_'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'
# modelMatrix            = out_path + 'modelMatrix'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'
# metrics_by_class       = out_path + 'metrics_By_Class'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'  
# confusionMatrix        = out_path + 'CM'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type) 



Input Binary File: ./data/rxjava_binaryBodyTitle_without_filter.csv
Output path      : ./goal format/rxjava_
dfTeste output   : ./goal format/rxjava_dfTeste.csv


[nltk_data] Downloading package stopwords to /Users/fd252/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# -*- coding: utf-8 -*-
# Importing required Library
import random
from sklearn.datasets import make_classification
from sklearn.neighbors import NearestNeighbors

In [4]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [5]:


def create_dataset(n_sample=1000):
    ''' 
    Create an unevenly distributed sample data set multilabel  
    classification using make_classification function
    
    args
    nsample: int, Number of sample to be created
    
    return
    X: pandas.DataFrame, feature vector dataframe with 10 features 
    y: pandas.DataFrame, target vector dataframe with 5 labels
    '''
    X, y = make_classification(n_classes=5, class_sep=2, 
                           weights=[0.1,0.025, 0.205, 0.008, 0.9], n_informative=3, n_redundant=1, flip_y=0,
                           n_features=10, n_clusters_per_class=1, n_samples=1000, random_state=10)
    y = pd.get_dummies(y, prefix='class')
    return pd.DataFrame(X), y

def get_tail_label(df):
    """
    Give tail label colums of the given target dataframe
    
    args
    df: pandas.DataFrame, target label df whose tail label has to identified
    
    return
    tail_label: list, a list containing column name of all the tail label
    """
    columns = df.columns
    n = len(columns)
    irpl = np.zeros(n)
    for column in range(n):
        #print(column)
        irpl[column] = df[columns[column]].value_counts()[1]
        #print(irpl[column])
    irpl = max(irpl)/irpl
    mir = np.average(irpl)
    tail_label = []
    for i in range(n):
        if irpl[i] > mir:
            tail_label.append(columns[i])
    return tail_label

def get_index(df):
    """
    give the index of all tail_label rows
    args
    df: pandas.DataFrame, target label df from which index for tail label has to identified
    
    return
    index: list, a list containing index number of all the tail label
    """
    tail_labels = get_tail_label(df)
    index = set()
    for tail_label in tail_labels:
        sub_index = set(df[df[tail_label]==1].index)
        index = index.union(sub_index)
    return list(index)

def get_minority_instace(X, y):
    """
    Give minority dataframe containing all the tail labels
    
    args
    X: pandas.DataFrame, the feature vector dataframe
    y: pandas.DataFrame, the target vector dataframe
    
    return
    X_sub: pandas.DataFrame, the feature vector minority dataframe
    y_sub: pandas.DataFrame, the target vector minority dataframe
    """
    #print(y)
    index = get_index(y)
    #print(index)
    X_sub = X[X.index.isin(index)].reset_index(drop = True)
    y_sub = y[y.index.isin(index)].reset_index(drop = True)
    return X_sub, y_sub

def nearest_neighbour(X):
    """
    Give index of 5 nearest neighbor of all the instance
    
    args
    X: np.array, array whose nearest neighbor has to find
    
    return
    indices: list of list, index of 5 NN of each element in X
    """
    nbs=NearestNeighbors(n_neighbors=3,metric='euclidean',algorithm='kd_tree').fit(X) #n_neighbors=3 mockito and n_neighbors=5 all
    euclidean,indices= nbs.kneighbors(X)
    return indices

def MLSMOTE(X,y, n_sample):
    """
    Give the augmented data using MLSMOTE algorithm
    
    args
    X: pandas.DataFrame, input vector DataFrame
    y: pandas.DataFrame, feature vector dataframe
    n_sample: int, number of newly generated sample
    
    return
    new_X: pandas.DataFrame, augmented feature vector data
    target: pandas.DataFrame, augmented target vector data
    """
    indices2 = nearest_neighbour(X)
    n = len(indices2)
    new_X = np.zeros((n_sample, X.shape[1]))
    target = np.zeros((n_sample, y.shape[1]))
    for i in range(n_sample):
        reference = random.randint(0,n-1)
        neighbour = random.choice(indices2[reference,1:])
        all_point = indices2[reference]
        nn_df = y[y.index.isin(all_point)]
        ser = nn_df.sum(axis = 0, skipna = True)
        target[i] = np.array([1 if val>2 else 0 for val in ser])
        ratio = random.random()
        gap = X.loc[reference,:] - X.loc[neighbour,:]
        new_X[i] = np.array(X.loc[reference,:] + ratio * gap)
    new_X = pd.DataFrame(new_X, columns=X.columns)
    target = pd.DataFrame(target, columns=y.columns)
    new_X = pd.concat([X, new_X], axis=0)
    target = pd.concat([y, target], axis=0)
    return new_X, target



In [6]:
#organizing data_frame to issue order
def organize():
    data_classes = pd.read_csv( binaryBodyTitle, header = 0, sep=';' )

    # OBS!!!!!
    #line 62 mockito was deleted wrong number of columns!
    #line 1923 rxjava was deleted wrong number of columns!
    
    
    ## Code implemented below before modifications by Jacob ##
    # del data_classes['prIssue']
    # del data_classes['issueTitle']
    # del data_classes['issueBody']
    
    # these renames are listed in the order that the original name appears in the list of headers
    # coming in from the binary file
    # data_classes.rename(columns={ 'pr': 'prNumber',                                   
    #                               'Title': 'prTitle',                                 
    #                               'Body': 'prBody',                                   
    #                               'issue': 'issueNumber',                             
    #                               'issueComments': 'prComments',                      
    #                               'issueTitleLink': 'issueTitle',                     
    #                               'issueBodyLink': 'issueBody',                       
    #                               'issueCommentsLink': 'issue_Comments',              
    #                               'Comments': 'prCodeReviewComments' }, inplace=True) 
    
    # categories = data_classes.columns.values.tolist()
    
    
    # this list must match the headers coming in from the binary file in name and order, including
    # with the renames above
    # data_classes = data_classes[[ 'pr', 'Util', 'NLP', 'APM', 'Network', 'DB', 'Interpreter', 'Logging', 
    #                               'Thread', 'DataStructure', 'i18n', 'DevOps', 'Logic',  
    #                               'Microservices', 'ML', 'Test', 'Search', 'IO', 'UI', 'Parser', 'Security',
    #                               'Cloud', 'BigData', 'App', 'GIS', 'Title', 'Body', 'prIssue', 'issue', 
    #                               'issueTitle', 'issueBody', 'issueComments', 'issueTitleLink', 'issueBodyLink', 
    #                               'issueCommentsLink', 'isPR', 'isTrain', 'commitMessage', 'Comments']]  


    ## Code above after modifications by Jacob ##
    # these renames are listed in the order that the original name appears in the list of headers
    # coming in from the binary file
    data_classes.rename(columns={ 'pr': 'prNumber',                                   
                                  'Title': 'prTitle',                                 
                                  'Body': 'prBody',                                   
                                  'issue': 'issueNumber',                             
                                  'issueComments': 'prComments',                      
                                  #'issueTitleLink': 'issueTitle',                     
                                  #'issueBodyLink': 'issueBody',                       
                                  'issueCommentsLink': 'issue_Comments',              
                                  'Comments': 'prCodeReviewComments',
                                'Data Structure': 'DataStructure',
                                'Big Data': 'BigData'}, inplace=True) 
    
    categories = data_classes.columns.values.tolist()
    
    
    # this list must match the headers coming in from the binary file in name and order, including
    # with the renames above
    #data_classes = data_classes[[ 'prNumber', 'Util', 'NLP', 'APM', 'Network', 'DB', 'Interpreter', 'Logging', 
    #                              'Thread', 'DataStructure', 'i18n', 'DevOps', 'Logic',
    #                              'Microservices', 'ML', 'Test', 'Search', 'IO', 'UI', 'Parser', 'Security',
    #                              'Cloud', 'BigData', 'App', 'GIS', 'prTitle', 'prBody', 'prIssue', 'issueNumber', 
    #                              'issueTitle', 'issueBody', 'prComments', 'issueTitleLink', 'issueBodyLink', 
    #                              'issue_Comments', 'isPR', 'isTrain', 'commitMessage', 'prCodeReviewComments']]    

    data_classes = data_classes[[ 'prNumber', 'Util', 'NLP', 'APM', 'Network', 'DB', 'Interpreter', 'Logging', 
                                  'DataStructure', 'i18n', 'DevOps', 'Logic',
                                  'Microservices', 'ML', 'Test', 'Search', 'IO', 'UI', 'Parser', 'Security',
                                  'Cloud', 'BigData', 'App', 'GIS', 'prTitle', 'prBody', 'prIssue', 'issueNumber', 
                                  'issueTitle', 'issueBody', 'prComments', 'issueTitleLink', 'issueBodyLink', 
                                  'issue_Comments', 'isPR', 'isTrain', 'commitMessage', 'prCodeReviewComments']]    

    #data_classes['issueNumber'] = data_classes['issueNumber'].astype('Int64')
    print('before filtering out empty classes',data_classes.shape)
    
    #find rows with parse error
    data_classes_error = data_classes.loc[pd.isnull(data_classes.loc[:,'Util'])]
    print('rows filtered out empty classes (parse error)',data_classes_error.shape)
    
    col_data_classes = len(data_classes.columns)

    if (len(data_classes_error) > 0):
        data_classes_fixed= data_classes_error.iloc[:,0].str.split(';', expand=True)
        print('rows fixed after new parse - empty classes (parse error)',data_classes_fixed.shape)

        col_data_classes_fixed = len(data_classes_fixed.columns)
        
        #removing rows with problems 
        data_classes.dropna(subset = ["Util"], inplace=True)
        print('after filtering out empty classes',data_classes.shape)
        
        print('len columns data_classes:',col_data_classes)
        print('len columns data_classes_fixed:',col_data_classes_fixed)
        
        if (col_data_classes == col_data_classes_fixed):
            
            names =['prNumber','DB','Interpreter','Logging','Thread','DataStructure','DevOps','i18n','Logic',
                    'Microservices','ML','Test','Search','IO','UI','Parser','Security','Cloud','BigData','App',
                    'GIS','Util','NLP','APM','Network','prTitle','prBody','prIssue','issueNumber','issueTitle',
                    'issueBody','prComments','issueTitleLink','issueBodyLink','issue_Comments','isPR','isTrain',
                    'commitMessage','prCodeReviewComments']                     
            data_classes_fixed.columns = names

            #drop data with error after parsing
            index_names = data_classes_fixed[ (data_classes_fixed['Util'] != '0') & (data_classes_fixed['Util'] != '1') |
                                     (data_classes_fixed['NLP'] != '0') & (data_classes_fixed['NLP'] != '1') |
                                     (data_classes_fixed['APM'] != '0') & (data_classes_fixed['APM'] != '1') |
                                     (data_classes_fixed['Network'] != '0') & (data_classes_fixed['Network'] != '1') |
                                     (data_classes_fixed['DB'] != '0') & (data_classes_fixed['DB'] != '1') |
                                     (data_classes_fixed['Interpreter'] != '0') & (data_classes_fixed['Interpreter'] != '1') |
                                     (data_classes_fixed['Logging'] != '0') & (data_classes_fixed['Logging'] != '1') |
                                     (data_classes_fixed['Thread'] != '0') & (data_classes_fixed['Thread'] != '1') |
                                     (data_classes_fixed['DataStructure'] != '0') & (data_classes_fixed['DataStructure'] != '1') |
                                     (data_classes_fixed['i18n'] != '0') & (data_classes_fixed['i18n'] != '1') |

                                     (data_classes_fixed['DevOps'] != '0') & (data_classes_fixed['DevOps'] != '1') |
                                     (data_classes_fixed['Logic'] != '0') & (data_classes_fixed['Logic'] != '1') |
                                   (data_classes_fixed['Microservices'] != '0') & (data_classes_fixed['Microservices'] != '1') |
                                   (data_classes_fixed['ML'] != '0') & (data_classes_fixed['ML'] != '1') |
                                   (data_classes_fixed['Test'] != '0') & (data_classes_fixed['Test'] != '1') |
                                   (data_classes_fixed['Search'] != '0') & (data_classes_fixed['Search'] != '1') |
                                   (data_classes_fixed['IO'] != '0') & (data_classes_fixed['IO'] != '1') |
                                   (data_classes_fixed['UI'] != '0') & (data_classes_fixed['UI'] != '1') |
                                   (data_classes_fixed['Parser'] != '0') & (data_classes_fixed['Parser'] != '1') |
                                 (data_classes_fixed['Security'] != '0') & (data_classes_fixed['Security'] != '1') |
                                 (data_classes_fixed['Cloud'] != '0') & (data_classes_fixed['Cloud'] != '1') |
                                 (data_classes_fixed['BigData'] != '0') & (data_classes_fixed['BigData'] != '1') |
                                 (data_classes_fixed['App'] != '0') & (data_classes_fixed['App'] != '1') |
                                 (data_classes_fixed['GIS'] != '0') & (data_classes_fixed['GIS'] != '1') 

                                    ].index

            # drop these given row
            # indexes from dataFrame
            data_classes_fixed.drop(index_names, inplace = True)
            print('data fixed after dropping parse fix errors',data_classes_fixed.shape)

            #back to float
            data_classes_fixed['Util'] = data_classes_fixed['Util'].astype(str).astype('Float64')
            data_classes_fixed['NLP'] = data_classes_fixed['NLP'].astype(str).astype('Float64')
            data_classes_fixed['APM'] = data_classes_fixed['APM'].astype(str).astype('Float64')
            data_classes_fixed['Network'] = data_classes_fixed['Network'].astype(str).astype('Float64')
            data_classes_fixed['DB'] = data_classes_fixed['DB'].astype(str).astype('Float64')
            data_classes_fixed['Interpreter'] = data_classes_fixed['Interpreter'].astype(str).astype('Float64')
            data_classes_fixed['Logging'] = data_classes_fixed['Logging'].astype(str).astype('Float64')
            data_classes_fixed['Thread'] = data_classes_fixed['Thread'].astype(str).astype('Float64')
            data_classes_fixed['DataStructure'] = data_classes_fixed['DataStructure'].astype(str).astype('Float64')
            data_classes_fixed['i18n'] = data_classes_fixed['i18n'].astype(str).astype('Float64')
            data_classes_fixed['DevOps'] = data_classes_fixed['DevOps'].astype(str).astype('Float64')
            data_classes_fixed['Logic'] = data_classes_fixed['Logic'].astype(str).astype('Float64')
            data_classes_fixed['Microservices'] = data_classes_fixed['Microservices'].astype(str).astype('Float64')
            data_classes_fixed['ML'] = data_classes_fixed['ML'].astype(str).astype('Float64')
            data_classes_fixed['Test'] = data_classes_fixed['Test'].astype(str).astype('Float64')
            data_classes_fixed['Search'] = data_classes_fixed['Search'].astype(str).astype('Float64')
            data_classes_fixed['IO'] = data_classes_fixed['IO'].astype(str).astype('Float64')
            data_classes_fixed['UI'] = data_classes_fixed['UI'].astype(str).astype('Float64')
            data_classes_fixed['Parser'] = data_classes_fixed['Parser'].astype(str).astype('Float64')
            data_classes_fixed['Security'] = data_classes_fixed['Security'].astype(str).astype('Float64')
            data_classes_fixed['Cloud'] = data_classes_fixed['Cloud'].astype(str).astype('Float64')
            data_classes_fixed['BigData'] = data_classes_fixed['BigData'].astype(str).astype('Float64')
            data_classes_fixed['App'] = data_classes_fixed['App'].astype(str).astype('Float64')
            data_classes_fixed['GIS'] = data_classes_fixed['GIS'].astype(str).astype('Float64')

            # appending fixed rows
            data_classes_new = data_classes.append(data_classes_fixed)
            print('after appending fixed rows',data_classes_new.shape)

            return data_classes_new
        
        else:
            print('fixing parse errors failed')

            return data_classes
    
    else:
        
        print('no parse errors found')
        return data_classes

In [7]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [65]:
data_classes = organize()

before filtering out empty classes (1981, 38)
rows filtered out empty classes (parse error) (0, 38)
no parse errors found


In [66]:
data_classes.shape

(1981, 38)

In [67]:
data_classes.head()


,prNumber,Util,NLP,APM,Network,DB,Interpreter,Logging,DataStructure,i18n,DevOps,Logic,Microservices,ML,Test,Search,IO,UI,Parser,Security,Cloud,BigData,App,GIS,prTitle,prBody,prIssue,issueNumber,issueTitle,issueBody,prComments,issueTitleLink,issueBodyLink,issue_Comments,isPR,isTrain,commitMessage,prCodeReviewComments
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,refactoring conventions,nan,NaN,0.0,refactoring conventions,NaN,NaN,NaN,NaN,NaN,1,1,refactoring for consistent implementation approachcombined observableextensions into o...,NaN
1,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,remove wrap functionality,we no longer need this as we ve merged the interface and abstract classes together an...,NaN,0.0,remove wrap functionality,we no longer need this as weve merged the interface and abstract classes together and ...,NaN,NaN,NaN,NaN,1,1,remove wrap functionalitywe no longer need this as weve merged the interface and abstr...,NaN
2,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,support multiple class types for language adaptors,nan,NaN,0.0,support multiple class types for language adaptors,NaN,NaN,NaN,NaN,NaN,1,1,support multiple class types for language adaptors,NaN
3,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,refactoring,- convert to rx.\* package names - rx.observables - rx.observables.operations (pr...,NaN,0.0,refactoring,- convert to rx\* package names - rxobservables - rxobservablesoperations (private p...,NaN,NaN,NaN,NaN,1,1,update language adaptors to rx* package,NaN
4,106,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,performance optimizations for dynamic function execution.,- memoize dynamic function constructions so executions are statically typed after a s...,NaN,0.0,performance optimizations for dynamic function execution,- memoize dynamic function constructions so executions are statically typed after a si...,test case:``` java public void runcompositiontest(integer[] values) { system...,NaN,NaN,NaN,1,1,performance optimizations for dynamic function execution- memoize dynamic function con...,NaN


In [12]:
#Filtering issues with PRs
def filtering(data_classes):
    
    print('before filtering out isTrain == 0',data_classes.shape)

    IssuePRDataset = data_classes[data_classes["isTrain"] == 0]
    
    print('after filtering out isTrain == 0',IssuePRDataset.shape)

    #invalid number of issue = NaN
    # IssuePRDataset = IssuePRDataset.drop([1805])

    categories = IssuePRDataset.columns.values.tolist()
    
    return categories, IssuePRDataset

In [68]:
categories, IssuePRDataset = filtering(data_classes)

before filtering out isTrain == 0 (1981, 38)
after filtering out isTrain == 0 (627, 38)


In [69]:
IssuePRDataset.tail()

,prNumber,Util,NLP,APM,Network,DB,Interpreter,Logging,DataStructure,i18n,DevOps,Logic,Microservices,ML,Test,Search,IO,UI,Parser,Security,Cloud,BigData,App,GIS,prTitle,prBody,prIssue,issueNumber,issueTitle,issueBody,prComments,issueTitleLink,issueBodyLink,issue_Comments,isPR,isTrain,commitMessage,prCodeReviewComments
1966,7170,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,2.x: introduce property rx2.scheduler.use-nanotime (#7154),fixes #7169,NaN,7154.0,2x: introduce property rx2scheduleruse-nanotime (#7154),fixes #7169,@akarnokd i had to create a new `schedulertest` class because on the 2x branch the pa...,3x feature: introduce abstract time source for scheduler impl,version:rxjava:309related:https://githubcom/reactivex/rxjava/issues/2943precondition:*...,you can override `now` when implementing `scheduler` and `worker` and delegate the oth...,1,0,2x: introduce property rx2scheduleruse-nanotime (#7154),NaN
1969,7191,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,3.x: add onsubscribe hook to parallelflowable operators,parallel operators were missing an rxjavaplugins.onsubscribe hook.,NaN,719.0,3x: add onsubscribe hook to parallelflowable operators,parallel operators were missing an `rxjavapluginsonsubscribe` hookresolves #7190,# [codecov](https://codecovio/gh/reactivex/rxjava/pull/7191?srcpr&elh1) report> mergin...,map doesnt work with random numbers,this may be specific to the scala adapter or a general issue im not sure it may also b...,this behavior is the expected behavior remove the line `var id ` and replace the line...,1,0,3x: add onsubscribe hook to parallelflowable operators,NaN
1970,7196,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,3.x: allow single.zip and maybe.zip result to be garbage collected,resolves #7195,NaN,719.0,3x: allow singlezip and maybezip result to be garbage collected,resolves #7195,@akarnokd sorry hit re-review accidentally :see_no_evil: || # [codecov](https://codec...,map doesnt work with random numbers,this may be specific to the scala adapter or a general issue im not sure it may also b...,this behavior is the expected behavior remove the line `var id ` and replace the line...,1,0,update maybeziparrayjava,NaN
1971,7203,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,3.x: fix schedulers.from to honor interruptibleworker across methods,the direct scheduling methods of the executorscheduler created via schedulers.from...,NaN,7201.0,3x: fix schedulersfrom to honor interruptibleworker across methods,the direct scheduling methods of the `executorscheduler` created via `schedulersfrom(e...,# [codecov](https://codecovio/gh/reactivex/rxjava/pull/7203?srcpr&elh1) report> mergin...,3x flowable#sample may complete on an interrupted thread,hi !using version 3010 and the following code:```javaint i 0while (true) { sys...,this is possible behavior because your completion races with the sampling when the ups...,1,0,3x: fix schedulersfrom to honor interruptibleworker across methods,NaN
1975,7212,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,3.x: compositeexception.printstacktrace to write directly into printstream/printwriter,i don t remember why the stacktrace it wasn t printed directly via the printstream/pr...,NaN,721.0,3x: compositeexceptionprintstacktrace to write directly into printstream/printwriter,i dont remember why the stacktrace it wasnt printed directly via the printstream/print...,# [codecov](https://codecovio/gh/reactivex/rxjava/pull/7212?srcpr&elh1) report> mergin...,add groupbyuntil operator to scala adapter,i have the following use case i create an observable that streams in timestamped data ...,cant you just use observables``` scaladef buffer(timespan: duration): observable[seq[t...,1,0,3x: compositeexceptionprintstacktrace to write directly into ps/pw,NaN


In [15]:
# RQ1.a - o quão sensível o resultado é em relação ao algoritmo? 
#vários algoritmos - BinaryRelevance
#todas as palavras, bootstrap, unigram 
#somente o título
def dataset_config(IssuePRDataset):
    # ORIGINAL
    # data_test1 = IssuePRDataset[['issueNumber','prNumber','issueTitle','Google Common', 
    #                              'Test', 'SO', 'IO', 'UI', 'Network', 'Security', 
    #                              'OpenOffice Documents', 'Database', 'Utils', 'PDF', 
    #                              'Logging', 'Latex']].copy()
    
    # WORKS WITH NEW INPUTS
    # data_test1 = IssuePRDataset[['issueNumber','prNumber','issueTitle', 'Test','IO', 'UI', 'Network', 'Security', 'Logging' ]].copy() 

    #data_test1 = IssuePRDataset[[ 'issueNumber','prNumber','issueTitle','issueBody', 'prTitle', 'prBody',
    #                         'issueTitleLink','issueBodyLink','commitMessage','prComments',
    #                         'Util', 
     #                             'NLP', 'APM', 'Network', 'DB', 'Interpreter',
     #                             'Logging', 'Thread', 'DataStructure', 'i18n', 
     #                             'DevOps', 'Logic', 'Microservices', 'ML',
     #                             'Test', 'Search', 'IO', 'UI', 'Parser', 'Security',
     #                             'Cloud', 'BigData', 'App', 'GIS' ]].copy()
    
    data_test1 = IssuePRDataset[[ 'issueNumber','prNumber','issueTitle','issueBody', 'prTitle', 'prBody',
                             'issueTitleLink','issueBodyLink','commitMessage','prComments',
                             'Util', 
                                  'NLP', 'APM', 'Network', 'DB', 'Interpreter',
                                  'Logging', 'DataStructure', 'i18n', 
                                  'DevOps', 'Logic', 'Microservices', 'ML',
                                  'Test', 'Search', 'IO', 'UI', 'Parser', 'Security',
                                  'Cloud', 'BigData', 'App', 'GIS' ]].copy()
    #print(type(data_test1))
    #data_test1['corpus'] = IssuePRDataset['issueTitle'] + IssuePRDataset['issueBody']
    data_test1["corpus"] = data_test1["issueTitle"].map(str)+" "+ data_test1["issueBody"].map(str)+" "+ data_test1["prComments"].map(str)

    # rxjava 2489 terms
    # mockito 598
    # presto 4
    # guava 1140
    # jabref 740
    
    #data_test1["corpus"] = data_test1["issueTitle"].map(str) + ' ' + data_test1["issueBody"].map(str) + ' ' + data_test1["prTitle"].map(str) + ' ' + data_test1["prBody"].map(str)
    # rxjava 3002 terms
    
    del data_test1["issueTitle"]
    del data_test1["issueBody"]
    del data_test1["prTitle"]
    del data_test1["prBody"]
    del data_test1["issueTitleLink"]
    del data_test1["issueBodyLink"]
    del data_test1["commitMessage"]
    del data_test1["prComments"]

    print('before filtering out empty corpus',data_test1.shape)
    data_test1['corpus'] = data_test1['corpus'].str.replace("nan",'')

    data_test1['corpus'].replace('', np.nan, inplace=True)
    data_test1.dropna(subset = ["corpus"], inplace=True)
    data_test1.dropna(subset = ["issueNumber"], inplace=True)
    
    print('after filtering out empty corpus',data_test1.shape)

    #removing utils because we won't to predict a so simple API that is basically used in all PRs
    #del data_test1["Util"]

    data_test1 = data_test1.reset_index(drop=True)
    
    return data_test1

In [72]:
data_test1 = dataset_config(IssuePRDataset)

before filtering out empty corpus (627, 26)
after filtering out empty corpus (627, 26)


In [73]:
data_test1.tail(200)

,issueNumber,prNumber,Util,NLP,APM,Network,DB,Interpreter,Logging,DataStructure,i18n,DevOps,Logic,Microservices,ML,Test,Search,IO,UI,Parser,Security,Cloud,BigData,App,GIS,corpus
427,5980.0,6052,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,#5980 made subscribeactual protected ### proposalchanged `subscribeactual` from `publi...
428,5806.0,6053,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2x: add maybe marble diagrams 06/21/a resolved the following diagrams of #5806:#### am...
429,605.0,6060,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,2x: fix concatmap{single|maybe} null emission on dispose race this pr fixes a bug in a...
430,5788.0,6074,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,add marble diagram to the singlenever method first of all im extremely grateful to ope...
431,5788.0,6075,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,add marble diagram to the singlefilter method hopefully this should tick off the `filt...
432,5788.0,6076,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,add marble diagrams to the singledelay method here are marbles for `delay` operator in...
433,5788.0,6077,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,add marble diagram for singlehide operator this should complete `hide` operator in #57...
434,3708.0,6079,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,2x: add completabletakeuntil(completable) operator this pr adds the missing dedicated ...
435,5788.0,6081,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,add marble diagrams for singlerepeat operators here are marbles for `repeat` `repeat(t...
436,5789.0,6083,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2x: add completable marble diagrams (07/17a) this pr adds the following marble diagram...


In [18]:
#preprocessing text

#We first convert the comments to lower-case 
#then use custom made functions to remove html-tags, punctuation and non-alphabetic characters from the TitleBody.

def clean_data(data_test1):
    if not sys.warnoptions:
        warnings.simplefilter("ignore")

    def cleanHtml(sentence):
        cleanr = re.compile('<.*?>')
        cleantext = re.sub(cleanr, ' ', str(sentence))
        return cleantext

    def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
        cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
        cleaned = re.sub(r'[\[|\]|<|>|@|.|,|)|(|\|/|`|\d|:|-|*]',r' ',cleaned)
        cleaned = cleaned.strip()
        cleaned = cleaned.replace("\n"," ")
        cleaned = cleaned.replace("-"," ")
        cleaned = cleaned.replace(r'[.|,|)|(|\|/]', '')
        cleaned = re.sub(r'^x ', r'', cleaned)
        return cleaned

    def keepAlpha(sentence):
        alpha_sent = ""
        for word in sentence.split():
            alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
            alpha_sent += alpha_word
            alpha_sent += " "
        alpha_sent = alpha_sent.strip()
        return alpha_sent
    
    def give_emoji_free_text(sentence):
        allchars = [str for str in sentence]
        emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
        clean_text = ' '.join([str for str in sentence.split() if not any(i in str for i in emoji_list)])

        return clean_text

    #function pra remover palavras com menos de 3 tokens

    data_test1['corpus'] = data_test1['corpus'].str.lower()
    data_test1['corpus'] = data_test1['corpus'].apply(cleanHtml)
    #data_test1['corpus'] = data_test1['corpus'].apply(cleanPunc)
    data_test1['corpus'] = data_test1['corpus'].apply(give_emoji_free_text)
    #data_test1['corpus'] = data_test1['corpus'].apply(keepAlpha)
    
    return data_test1


In [74]:
data_test1 = clean_data(data_test1)

In [75]:
data_test1.head()

,issueNumber,prNumber,Util,NLP,APM,Network,DB,Interpreter,Logging,DataStructure,i18n,DevOps,Logic,Microservices,ML,Test,Search,IO,UI,Parser,Security,Cloud,BigData,App,GIS,corpus
0,76.0,138,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,implemented sequenceequal operator fixes issue #76since there is no direct analogue of...
1,125.0,146,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,merge of pull #125 for issue #87 operator takewhile manually merging https://githubcom...
2,87.0,146,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,merge of pull #125 for issue #87 operator takewhile manually merging https://githubcom...
3,18.0,156,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,implemented toiterable operation for issue #18this would allow to implement other bloc...
4,164.0,165,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,fixed bug in ranges fixed issue #164 [rxjava-pull-requests #3](https://netflixossciclo...


In [50]:
def remove_columns(data_classifier):
    if proj_name == 'presto' or proj_name == 'presto_2': 
        print("================Presto-\nSUM OF COLUMNS\n=========================")

        del data_classifier['NLP']
        del data_classifier['APM']
        del data_classifier['Logging']
        #del data_classifier['Thread']
        del data_classifier['DataStructure']
        del data_classifier['Microservices']
        del data_classifier['ML']
        del data_classifier['Logic']
        del data_classifier['Search']
        del data_classifier['UI']
        del data_classifier['Cloud']
        del data_classifier['Parser']
        del data_classifier['Security']
        del data_classifier['BigData']
        del data_classifier['App']
        del data_classifier['GIS']
        #print(data_classifier)
    if proj_name == 'guava' or proj_name == 'guava_2' :
        print("================Guava-\nSUM OF COLUMNS\n=========================")

        del data_classifier['NLP']
        del data_classifier['APM']
        del data_classifier['Logging']
        del data_classifier['DataStructure']
        del data_classifier['DevOps']
        del data_classifier['Microservices']
        del data_classifier['ML']
        del data_classifier['Search']
        del data_classifier['Parser']
        del data_classifier['Cloud']
        del data_classifier['BigData']
        del data_classifier['App']
        del data_classifier['GIS']
    
    if proj_name == 'jabref' or proj_name == 'jabref_2': 
        print("================JabRef-\nSUM OF COLUMNS\n=========================")

        #del data_classifier['Thread']
        del data_classifier['DataStructure']
        del data_classifier['Microservices']
        del data_classifier['ML']
        del data_classifier['Cloud']
        del data_classifier['BigData']
        del data_classifier['GIS']

    if proj_name == 'jabref50' or proj_name == 'jabref50_2': 
        print("================JabRef-\nSUM OF COLUMNS\n=========================")

        #del data_classifier['Thread']
        del data_classifier['DataStructure']
        del data_classifier['Microservices']
        del data_classifier['ML']
        del data_classifier['Search']
        del data_classifier['Parser']                        
        del data_classifier['Cloud']
        del data_classifier['BigData']
        del data_classifier['GIS']

    if proj_name == 'mockito' or proj_name == 'mockito_2': 
        print("================Mockito-\nSUM OF COLUMNS\n=========================")

        del data_classifier['NLP']
        del data_classifier['APM']
        del data_classifier['DB']
        del data_classifier['Logging']
        #del data_classifier['Thread']
        del data_classifier['DataStructure']  
        del data_classifier['DevOps'] 
        del data_classifier['Microservices']
        del data_classifier['ML']
        del data_classifier['Search']
        del data_classifier['UI']
        del data_classifier['Parser']
        del data_classifier['Cloud']
        del data_classifier['BigData']
        del data_classifier['App']
        del data_classifier['GIS']

    if proj_name == 'rxjava' or proj_name == 'rxjava_2': 
        print("================RxJava-\nSUM OF COLUMNS\n=========================")
        del data_classifier['NLP']
        del data_classifier['APM']
        del data_classifier['DB']
        #del data_classifier['Interpreter']
        del data_classifier['Logging']
        #del data_classifier['Thread']
        del data_classifier['DataStructure']
        del data_classifier['DevOps']
        #del data_classifier['Logic']
        del data_classifier['Microservices']
        del data_classifier['ML']
        del data_classifier['Search']
        del data_classifier['UI']
        del data_classifier['Cloud']
        del data_classifier['Parser']
        del data_classifier['Security']
        del data_classifier['BigData']
        #del data_classifier['App']
        del data_classifier['GIS']
    return data_classifier

In [76]:
data_classifier = remove_columns(data_test1)


================RxJava-
SUM OF COLUMNS


In [77]:
data_classifier.head()

,issueNumber,prNumber,Util,Network,Interpreter,i18n,Logic,Test,IO,App,corpus
0,76.0,138,1,0,0,0,0,0,1,0,implemented sequenceequal operator fixes issue #76since there is no direct analogue of...
1,125.0,146,1,0,0,0,0,0,1,0,merge of pull #125 for issue #87 operator takewhile manually merging https://githubcom...
2,87.0,146,1,0,0,0,0,0,1,0,merge of pull #125 for issue #87 operator takewhile manually merging https://githubcom...
3,18.0,156,1,0,0,0,0,0,1,0,implemented toiterable operation for issue #18this would allow to implement other bloc...
4,164.0,165,1,0,0,0,0,0,1,0,fixed bug in ranges fixed issue #164 [rxjava-pull-requests #3](https://netflixossciclo...


In [78]:
data_classifier.columns

Index(['issueNumber', 'prNumber', 'Util', 'Network', 'Interpreter', 'i18n',
       'Logic', 'Test', 'IO', 'App', 'corpus'],
      dtype='object')

In [79]:
data_classifier.shape

(627, 11)

In [80]:
data_classifier.to_csv('./goal format/'+ proj_name + 'data_classescleanstopwds.csv', encoding='utf-8', sep=',', index=False)


In [56]:
import datetime
#number of shuffles (folds)
splits=10

#defining paths
# loggingFile = './experiment//TitleBody3GramTFIDF.txt'
# binaryBodyTitle = './experiment/binaryBodyTitle.csv'
template = output_root_dir + proj_name + '/' + 'dfTeste.csv'

configurationTFIDFList = [(1,1)]
#configurationTFIDFList = [(1,1),(2,2),(3,3),(4,4)]
#configurationTFIDFList = [(2,2),(3,3),(4,4)]
#num_featureList = [25,50,100,500,1000,2500,5000]
#num_featureList = [90,179,358,716,895,1432,2864,5728]
#num_featureList = [740] #jabref body 739 v2
#num_featureList = [1140] #guava body  1141 v2
#num_featureList = [4] #presto body 4 v2
#num_featureList = [598] #mockito body 597 v2
#num_featureList = [2489] #rxjava body 2436 v2
#num_featureList = [974] #jabref50 body

#num_featureList = [228] #jabref title
#num_featureList = [6] #guava title
#num_featureList = [7] #presto title
#num_featureList = [216] #mockito title
#num_featureList = [800] #rxjava title
num_featureList = [838] #jabref50 title

#num_featureList = [794] #jabref title+body
#num_featureList = [1143] #guava title+body
#num_featureList = [10] #presto title+body
#num_featureList = [654] #mockito title+body
#num_featureList = [2615] #rxjava title+body
#num_featureList = [2818] #jabref50 title+body

#num_featureList = [1336] #jabref title+body+comments
#num_featureList = [1204] #guava title+body+comments
#num_featureList = [10] #presto title+body+comments
#num_featureList = [974] #mockito title+body+comments
#num_featureList = [3269] #rxjava title+body+comments
#num_featureList = [5622] #jabref50 title+body+comments


size_testList = [0.2]
#size_testList = [0.2,0.3,0.4]
stop_wordList = ["Yes"]
#test_typeList = ["RandomForest","DecisionTree","LogisticRegression","MLPClassifier","MLkNN"]
#test_typeList = ["RandomForest","DecisionTree","MLPClassifier","MLkNN"]
#test_typeList = ["RandomForest","DecisionTree","MLPClassifier"]
test_typeList = ["RandomForest"]

#examples
#configurationTFIDFList = [(1,1),(2,2)]
#num_featureList = [25,50]
#size_testList = [0.2,0.3]
#stop_wordList = ["Yes","No"]
#test_typeList = ["RandomForest","DecisionTree"]

configurationTFIDF=(1,1)
num_feature=895

size_test=0.2

#stop_word = stop_wordList[i]
stop_word = "Yes"

test_type = "RandomForest"
# dont forget to have the file dfTeste.csv ready in the files folder only with the header

def __main__():
    
    print("----------------") 
    now = datetime.datetime.now()

    
    print ("Current date and time : ")
    print (now.strftime("%Y-%m-%d %H:%M:%S"))

    print(proj_name)
    print("----------------") 

    # getting length of list 
    lengthT = len(configurationTFIDFList) 
    lengthF = len(num_featureList) 
    lengthS = len(size_testList)
    lengthY = len(test_typeList)
    
    # Iterating the index 
    # same as 'for i in range(len(list))' 
    for t in range(lengthT): 
        print("----------------") 
        print(configurationTFIDFList[t])
        print("----------------") 

        for f in range(lengthF): 
            print("----------------") 
            print(configurationTFIDFList[t])
            print(num_featureList[f]) 
            print("----------------") 


            for s in range(lengthS):
                print("----------------") 
                print(configurationTFIDFList[t]) 
                print(num_featureList[f]) 
                print(size_testList[s]) 
                print("----------------") 

                for y in range(lengthY):
                    print("----------------") 
                    print(configurationTFIDFList[t]) 
                    print(num_featureList[f]) 
                    print(size_testList[s]) 
                    print(test_typeList[y]) 
                    print("----------------") 

                    configurationTFIDF=configurationTFIDFList[t]
                    num_feature=num_featureList[f]

                    size_test=size_testList[s]

                    #stop_word = stop_wordList[i]
                    stop_word = "Yes"

                    test_type = test_typeList[y]
                    
                    # tfIDFoutputFile = './experiment/tfIDFoutputFile'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'
                    # classifierFeatureInput='./experiment/train_file_test'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'
                    # termFrequencyTop50 = './experiment/termFrequencyTop50'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'
                    # predictions_result = './experiment/predict_file_'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'
                    # probabilities_result = './experiment/probability_file_'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'
                    # modelMatrix = './experiment/modelMatrix'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'
                    # metrics_by_class = './experiment/metrics_By_Class'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'  
                    # confusionMatrix = './experiment/CM'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type) 
                    tfIDFoutputFile        = out_path + 'tfIDFoutputFile'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'
                    classifierFeatureInput = out_path + 'train_file_test'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'
                    termFrequencyTop50     = out_path + 'termFrequencyTop50'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'
                    predictions_result     = out_path + 'predict_file_'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'
                    probabilities_result   = out_path + 'probability_file_'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'
                    modelMatrix            = out_path + 'modelMatrix'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'
                    metrics_by_class       = out_path + 'metrics_By_Class'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type)+'.csv'  
                    confusionMatrix        = out_path + 'CM'+ str(configurationTFIDF) + str(num_feature) + stop_word + str(size_test)+ str(test_type) 

                    data_classes = organize()
                    print("data_classes", data_classes.columns )
                    print("data_classes", data_classes.shape )
                    categories, IssuePRDataset = filtering(data_classes)
                    print("IssuePRdataset:", IssuePRDataset.columns )
                    print("IssuePRdataset:", IssuePRDataset.shape )
                    data_test1 = dataset_config(IssuePRDataset)
                    print("data test1 after config:", data_test1.columns )
                    print("data test1 after config:", data_test1.shape )
                    #data_test1.to_csv('rxjava_data_classesunclean.csv', encoding='utf-8', sep=',', index=False)
                    data_test1 = clean_data(data_test1)
                    print("data test1 after clean:", data_test1.columns )
                    print("data test1 after clean:", data_test1.shape )

                    #STOP CLEANING HERE FOR FAST BERT, SAVE CSV AND EXIT
                    data_test1.to_csv('./goal format/'+ proj_name + 'data_classescleans.csv', encoding='utf-8', sep=',', index=False)
                    #sys.exit()

                    #re_stop_words = remove_stop_words()
                    
                    #data_test1['corpus'] = data_test1['corpus'].apply(removeStopWords, re_stop_words=re_stop_words)
                    #data = data_test1
#                    print('2',data_test1['corpus'])
                    #data_test1.to_csv('rxjava_data_classescleannostopwds.csv', encoding='utf-8', sep=',', index=False)
                    
                    #data_test1 = apply_stem(data)
                    
                   # data_test1.to_csv('rxjava_data_classescleanstemmed.csv', encoding='utf-8', sep=',', index=False)
                        
                #    docs, a = analyze_top(data_test1, termFrequencyTop50)

                   # print('num features discivered automaticaly', len(a))
                    #data_test1 = remove_columns(data_test1)
                    data_classifier = data_test1
                    print("PROJ NAME: ", proj_name)
                    """if proj_name == 'rxjava' or proj_name == 'rxjava_2': 
                        print("================RxJava-\nSUM OF COLUMNS\n=========================")
                        del data_classifier['NLP']
                        del data_classifier['APM']
                        del data_classifier['DB']
                        del data_classifier['Interpreter']
                        del data_classifier['Logging']
                        #del data_classifier['Thread']
                        del data_classifier['DataStructure']
                        del data_classifier['DevOps']
                        del data_classifier['Logic']
                        del data_classifier['Microservices']
                        del data_classifier['ML']
                        del data_classifier['Search']
                        del data_classifier['UI']
                        del data_classifier['Cloud']
                        del data_classifier['Parser']
                        del data_classifier['Security']
                        del data_classifier['BigData']
                        del data_classifier['App']
                        del data_classifier['GIS']"""
                   # data_test1.to_csv('rxjava_data_classescleanstemmedcolsremoved.csv', encoding='utf-8', sep=',', index=False)
                    #Finished making csvs, exit.
                    #sys.exit()
#                    print('3',data_test1['corpus'])
                    #run_tf_idf(data_test1, configurationTFIDF, num_feature, tfIDFoutputFile)
                    #run_tf_idf(data_test1, configurationTFIDF, len(a), tfIDFoutputFile)
#                    print('4',data_test1['corpus'])
                    

                    
                    
                    #data_classifier, categories = merging(data_test1, tfIDFoutputFile)     
                    
                    # some rows are with nan. df = df[df['my_var'].notna()] ?
                    # or should we change by blank spaces?

                    print("================COLUMNS BEFORE DELETION\n=========================")
                    print(data_classifier.shape)
                    print(list(data_classifier.columns))
                    no_occurences = data_classifier.loc[:,'Util':'GIS'].sum()
                    #print(type(no_occurences))
                    print(no_occurences)  #df.loc[:, 'Score'].sum()
  
                    data_classifier = remove_columns(data_classifier)
                    print("data test1 after remove cols:", data_classifier.columns )
                    print("data test1 after remove cols:", data_classifier.shape )

                    #STOP CLEANING HERE FOR FAST BERT, SAVE CSV AND EXIT (AFTER REMOVE)
                    data_classifier.to_csv('./goal format/'+ proj_name + 'data_classescleansremove.csv', encoding='utf-8', sep=',', index=False)
                    #sys.exit()
                   

                    print("================COLUMNS AFTER DELETION\n=========================")
                    print(data_classifier.shape)
                    
                    #SMOTE)
                    print(list(data_classifier.columns))
                    
                    #SMOTE
                    limit = len(data_classifier.columns)
                    
                    if proj_name == 'jabref50' or proj_name == 'jabref50_2':
                        our_X = data_classifier.iloc[:,19:limit] # TF-IDF probabilities jabref
                        our_y = data_classifier.iloc[:,2:17] # Expert labels jabref
                    
                    if proj_name == 'jabref' or proj_name == 'jabref_2':
                        our_X = data_classifier.iloc[:,21:limit] # TF-IDF probabilities jabref
                        our_y = data_classifier.iloc[:,2:19] # Expert labels jabref
                    
                    if proj_name == 'guava' or proj_name == 'guava_2': 
                        our_X = data_classifier.iloc[:,15:limit] # TF-IDF probabilities guava
                        our_y = data_classifier.iloc[:,2:13] # Expert labels guava

                    if proj_name == 'presto' or proj_name == 'presto_2': 
                        our_X = data_classifier.iloc[:,12:limit] # TF-IDF probabilities guava
                        our_y = data_classifier.iloc[:,2:10] # Expert labels guava
                        
                    if proj_name == 'mockito' or proj_name == 'mockito_2': 
                        our_X = data_classifier.iloc[:,12:limit] # TF-IDF probabilities guava
                        our_y = data_classifier.iloc[:,2:10] # Expert labels guava

                    if proj_name == 'rxjava' or proj_name == 'rxjava_2': 
                        #our_X = data_classifier.iloc[:,9:limit] # TF-IDF probabilities rxjava
                        our_X = data_classifier.iloc[:,10:limit] # corpus rxjava
                        our_y = data_classifier.iloc[:,2:10] # Expert labels rxjava


                    print("================our_y AFTER DELETION\n=========================")
                    print(our_y.shape)
                    print(our_y.columns)
                    print("================our_X AFTER DELETION\n=========================")
                    print(our_X.shape)
                    print(our_X.columns)

                    dataset = pd.concat([our_y,our_X], axis=1)                    
                    print("dataset ready for smote:",dataset.shape)

                    if proj_name != 'presto': 

                        X_sub, y_sub = get_minority_instace(our_X, our_y)   #Getting minority instance of that datframe


                        X_res,y_res =MLSMOTE(X_sub, y_sub, 100)     #Applying MLSMOTE to augment the dataframe

                        y_X_res = pd.concat([y_res,X_res], axis=1)
                        #y_X_res.shape

                        dataset_smote = pd.concat([dataset, y_X_res], axis=0)
                        
                    else:
                        
                        dataset_smote = dataset

                    print("================SMOTE \n=========================")

                    print(dataset_smote.shape)
                    dataset_smote.to_csv('./goal format/'+ proj_name + 'data_classescleanstopwds_smote.csv', encoding='utf-8', sep=',', index=False)
                    sys.exit()
                    #run_split(data_classifier, test_type, confusionMatrix, modelMatrix, metrics_by_class, configurationTFIDF ,num_feature ,stop_word ,size_test)
                    #run_split(dataset_smote, test_type, confusionMatrix, modelMatrix, metrics_by_class, configurationTFIDF ,num_feature ,stop_word ,size_test, proj_name)
                    
    print("----------------") 
    print( "Operations complete!")
    now = datetime.datetime.now()

    
    print ("Current date and time : ")
    print (now.strftime("%Y-%m-%d %H:%M:%S"))
    print("----------------") 



In [81]:
limit = len(data_classifier.columns)

if proj_name == 'jabref50' or proj_name == 'jabref50_2':
    our_X = data_classifier.iloc[:,19:limit] # TF-IDF probabilities jabref50
    our_y = data_classifier.iloc[:,2:17] # Expert labels jabref50

if proj_name == 'jabref' or proj_name == 'jabref_2':
    our_X = data_classifier.iloc[:,21:limit] # TF-IDF probabilities jabref
    our_y = data_classifier.iloc[:,2:19] # Expert labels jabref

if proj_name == 'guava' or proj_name == 'guava_2': 
    our_X = data_classifier.iloc[:,15:limit] # TF-IDF probabilities guava
    our_y = data_classifier.iloc[:,2:13] # Expert labels guava

if proj_name == 'presto' or proj_name == 'presto_2': 
    our_X = data_classifier.iloc[:,12:limit] # TF-IDF probabilities guava
    our_y = data_classifier.iloc[:,2:10] # Expert labels guava

if proj_name == 'mockito' or proj_name == 'mockito_2': 
    our_X = data_classifier.iloc[:,12:limit] # TF-IDF probabilities mockito
    our_y = data_classifier.iloc[:,2:10] # Expert labels mockito

if proj_name == 'rxjava' or proj_name == 'rxjava_2': 
    #our_X = data_classifier.iloc[:,9:limit] # TF-IDF probabilities rxjava
    our_X = data_classifier.iloc[:,10:limit] # corpus rxjava
    our_y = data_classifier.iloc[:,2:10] # Expert labels rxjava


In [82]:
data_classifier.columns

Index(['issueNumber', 'prNumber', 'Util', 'Network', 'Interpreter', 'i18n',
       'Logic', 'Test', 'IO', 'App', 'corpus'],
      dtype='object')

In [83]:
limit

11

In [84]:
our_X = data_classifier.iloc[:,10:limit] # TF-IDF probabilities guava
our_X

,corpus
0,implemented sequenceequal operator fixes issue #76since there is no direct analogue of...
1,merge of pull #125 for issue #87 operator takewhile manually merging https://githubcom...
2,merge of pull #125 for issue #87 operator takewhile manually merging https://githubcom...
3,implemented toiterable operation for issue #18this would allow to implement other bloc...
4,fixed bug in ranges fixed issue #164 [rxjava-pull-requests #3](https://netflixossciclo...
5,reimplementation of concat improved handling of observable >the old version required a...
6,schedulers (merge of pull #199) manual merge of https://githubcom/netflix/rxjava/pull/...
7,"interval i have attempted to implement the ""interval"" operator (#55) here as far as i ..."
8,trying to extend the scheduler interface according to the comments at ive tried to ext...
9,implemented sample operation now that `operationinterval` is in `operationsample` (see...


In [85]:
our_y = data_classifier.iloc[:,2:10] # Expert labels guava
our_y

,Util,Network,Interpreter,i18n,Logic,Test,IO,App
0,1,0,0,0,0,0,1,0
1,1,0,0,0,0,0,1,0
2,1,0,0,0,0,0,1,0
3,1,0,0,0,0,0,1,0
4,1,0,0,0,0,0,1,0
5,1,0,0,0,0,0,1,0
6,1,0,0,0,0,0,1,0
7,1,0,0,0,0,0,1,0
8,1,0,0,0,0,0,1,0
9,1,0,0,0,0,0,1,0


In [86]:
X_sub, y_sub = get_minority_instace(our_X, our_y)   #Getting minority instance of that datframe




In [87]:
y_sub.head()

,Util,Network,Interpreter,i18n,Logic,Test,IO,App
0,1,0,1,0,0,0,1,0
1,1,0,1,0,0,1,1,1
2,1,0,1,0,0,1,1,1
3,1,0,1,0,0,1,1,1
4,1,0,1,0,0,0,1,0


In [88]:
X_sub.head()

,corpus
0,concurrency fixes for rxringbuffer & merge possible fix for #1420i was unable to exact...
1,subscriberrequest should throw exception if negative request made as per discussion in...
2,subscriberrequest should throw exception if negative request made as per discussion in...
3,subscriberonstart requests should be additive (and check for overflow) as per discussi...
4,takewhile: dont unsubscribe downstream fixes #2647 issue with takewhile


In [89]:
X_res,y_res =MLSMOTE(X_sub, y_sub, 100)     #Applying MLSMOTE to augment the dataframe




ValueError: could not convert string to float: 'concurrency fixes for rxringbuffer & merge possible fix for #1420i was unable to exactly reproduce the issue but was able to create a possibly related one this fixes itit also simplifies the logic and code somewhat at a possible small performance cost for these:```rooperatormergeperfmerge1syncstreamofn 1000 thrpt 5 59407716 2073945 ops/srooperatormergeperfmerge1syncstreamofn 1000000 thrpt 5 66557 2693 ops/srooperatormergeperfmergensyncstreamsofn 1000 thrpt 5 59293 5046 ops/s```'

In [ ]:
y_X_res = pd.concat([y_res,X_res], axis=1)
#y_X_res.shape

dataset_smote = pd.concat([dataset, y_X_res], axis=0)

In [57]:
__main__()


----------------
Current date and time : 
2021-12-01 14:54:37
rxjava
----------------
----------------
(1, 1)
----------------
----------------
(1, 1)
838
----------------
----------------
(1, 1)
838
0.2
----------------
----------------
(1, 1)
838
0.2
RandomForest
----------------
before filtering out empty classes (1981, 38)
rows filtered out empty classes (parse error) (0, 38)
no parse errors found
data_classes Index(['prNumber', 'Util', 'NLP', 'APM', 'Network', 'DB', 'Interpreter',
       'Logging', 'DataStructure', 'i18n', 'DevOps', 'Logic', 'Microservices',
       'ML', 'Test', 'Search', 'IO', 'UI', 'Parser', 'Security', 'Cloud',
       'BigData', 'App', 'GIS', 'prTitle', 'prBody', 'prIssue', 'issueNumber',
       'issueTitle', 'issueBody', 'prComments', 'issueTitleLink',
       'issueBodyLink', 'issue_Comments', 'isPR', 'isTrain', 'commitMessage',
       'prCodeReviewComments'],
      dtype='object')
data_classes (1981, 38)
before filtering out isTrain == 0 (1981, 38)
after filt

ValueError: could not convert string to float: 'concurrency fixes for rxringbuffer & merge possible fix for #1420i was unable to exactly reproduce the issue but was able to create a possibly related one this fixes itit also simplifies the logic and code somewhat at a possible small performance cost for these:```rooperatormergeperfmerge1syncstreamofn 1000 thrpt 5 59407716 2073945 ops/srooperatormergeperfmerge1syncstreamofn 1000000 thrpt 5 66557 2693 ops/srooperatormergeperfmergensyncstreamsofn 1000 thrpt 5 59293 5046 ops/s```'